In [1]:
import pandas as pd
import re
# Open a file: file
sent_table = pd.read_csv('data/corpus.txt', delimiter='|', usecols=['text'])
print(sent_table)


for i in sent_table.index.values:
    print(sent_table.iloc[i]['text']);
    #sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].replace('''O ''', '''Oh''')
    sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].replace('''`''', ''' \'''')
    sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].replace(''' ` ''', '''\'''')
    sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].replace(''' `''', '''\'''')
    sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].replace(''' \'''','''\'''')
    sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].replace('''\'d''','''ed''')
    #sent_table.iloc[i]['text'] = re.sub(r'''[^\w\d'\-\s]+''','',sent_table.iloc[i]['text']);
    sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].strip();
    
    
#%%
def uniq(input):
  output = []
  for x in input:
    if x not in output:
      output.append(x)
  return output

def recursive_len(item):
    if type(item) == list:
        return sum(recursive_len(subitem) for subitem in item)
    else:
        return 1

import nltk
import numpy as np
import time
from datetime import datetime

num_figs=10
max_fig_length=18
base_path = 'data/rhet/repitition/'
base_path_output = 'data/rhet/output/'
corpus = 'sonnets'

fig_names = ["Anadiplosis", "Anaphora", "Antimetabole", "Conduplicatio", "Epanalepsis", "Epistrophe", "Epizeuxis", "Ploce", "Polysyndeton", "Symploce"]

fig_descs = \
    [
        "Repetition of the ending word or phrase from the previous clause at the beginning of the next.",
        "Repetition of a word or phrase at the beginning of successive phrases or clauses.",
        "Repetition of words in reverse grammatical order.",
        "The repetition of a word or phrase.",
        "Repetition at the end of a clause of the word or phrase that began it.",
        "Repetition of the same word or phrase at the end of successive clauses.",
        "Repetition of a word or phrase with no others between.",
        "The repetition of word in a short span of text for rhetorical emphasis.",
        "\"Excessive\" repetition of conjunctions between clauses.",
        "Repetition of a word or phrase at the beginning, and of another at the end, of successive clauses."
    ]

rep_figs = ["sonnet_anadiplosis.csv", "sonnet_anaphora.csv", "sonnet_antimetabole.csv",
"sonnet_conduplicatio.csv", 
            "sonnet_epanalepsis.csv", "sonnet_epistrophe.csv", "sonnet_epizeuxis.csv",
"sonnet_ploce.csv", 
            "sonnet_polysyndeton.csv", "sonnet_symploce.csv"]

from py_files.Limericks import Limerick_Generate
lg = Limerick_Generate(model_dir='/gpt2/models/345M',model_name='345M')
n=200
banned_set = ['foo']
for nfig in range(0,num_figs):
    
    cur_fig_name = fig_names[nfig]
    cur_fig_desc = fig_descs[nfig]
    cur_fig_rhet_file = rep_figs[nfig]

    rhet_table = pd.read_csv(base_path+cur_fig_rhet_file)
    timestamp = int(time.mktime(datetime.now().timetuple()))
    out_name = base_path_output + corpus + "_" + cur_fig_name + "_" + str(timestamp) +".txt"
    f = open(out_name,"w+")
    
    f.write("FIGURE: %s\n" % cur_fig_name)
    f.write("DESCRIPTIOM: %s\n" % cur_fig_desc)
    f.write("CORPUS: %s\n\n" % corpus)
    fig_ids = rhet_table.figure_id.unique()
    #print(fig_ids)
    for id in fig_ids:
       figure = rhet_table.loc[rhet_table['figure_id'] == id]
       rep_word = []
       sentence_ids = figure['sentence_id'].values
       sentence_ids = uniq(sentence_ids)
       if(not sorted(sentence_ids) == list(range(min(sentence_ids), max(sentence_ids)+1))): continue 
       print(sentence_ids)
       fig_words = uniq([x.lower().replace('''\'d''', '''ed''') for x in
list(figure['word'].values)])
       fig_words_dict = dict([[fig_words[y-1], y] for y in range(1, len(fig_words)+1)])
       #print(fig_words_dict)
        
       orig_text = [ nltk.word_tokenize(sent_table.iloc[i]['text']) for i in sentence_ids]
       pattern = '^M{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$'
       roman = []
       for j in range(0,len(orig_text)):
           for k in range(0,len(orig_text[j])):
               if(re.search(pattern, orig_text[j][k])):
                   roman.append([j,k])
       
       #for l in range(0,len(roman)):
           #del orig_text[roman[l][0]][roman[l][1]]
    
       orig_text = [[orig_text[i][j].lower() for j in range(0,len(orig_text[i]))] for i
in range(0,len(orig_text))]
       if(len(sentence_ids) == 1 and recursive_len(orig_text) <= 18 and recursive_len(orig_text) > 1):
           tagged = [nltk.pos_tag(orig_text[i]) for i in  range(0,len(orig_text))]
           #print(orig_text)
           pos_templates = [[tagged[i][j][1] for j in range(0,len(tagged[i]))] for i in
range(0,len(tagged))]
           #print(pos_templates)
           
           assert(np.shape(orig_text) == np.shape(pos_templates))
           fig_words_count_dict = dict([[fig_words[y], 0] for y in range(0,
len(fig_words))])
        
           rep_templates = [['0' for j in range(0,len(orig_text[i]))] for i in
range(0,len(orig_text))]
           for j in range(0,len(orig_text)):
               for k in range(0,len(orig_text[j])):
                   wordc = orig_text[j][k]
                   for word_rep in list(fig_words_dict.keys()):
                       if(word_rep in wordc):
                           fig_words_count_dict[word_rep] = fig_words_count_dict[word_rep]+1
                           rep_templates[j][k] = str(fig_words_dict[word_rep]) + str(fig_words_count_dict[word_rep])
           
           f.write("Orig. Text:\n")
           orig_line = ", ".join([' '.join(orig_text[i]) for i in
range(0,len(orig_text))])
           print(orig_line)
           f.write(orig_line)
           f.write("\n")
           f.write("Rep. Words:\n")
           f.writelines(', '.join(fig_words_dict))
           f.write("\n")
           f.write("POS Template:\n")
           f.writelines(str(pos_templates))
           f.write("\n")
           f.write("Rep Template:\n")
           f.writelines(str(rep_templates))
           f.write("\n")
           f.write("Sample Output:\n")
           
           print(rep_templates[0])
           print(pos_templates[0])
           
           prompt = lg.enc.encode(orig_line + ",")
           try:
               new_sentences = lg.gen_line_gpt_cc(cctemplate=rep_templates[0], w=None, encodes=prompt, default_template=pos_templates[0], banned_set=banned_set,search_space=n)
               sents = [new_sentences[j][0] for j in range(len(new_sentences))]
               sent_done = [' '.join(sents[j]) for j in range(len(new_sentences))]
               f.write(" \n".join(sent_done))
 
           except Exception:
               pass
           f.write("\n-----------------------------------------------------------------------------------------------------------------------------------------------------\n") 
           print(rep_templates)     
  
   #for index, row in figure.iterrows():
    f.close()
   #source = sentences
   # while(!re.search(exp, source)
        



                                                   text
0           From fairest creatures we desire increase ,
1         That thereby beauty 's rose might never die ,
2             But as the riper should by time decease ,
3               His tender heir might bear his memory :
4      But thou , contracted to thine own bright eyes ,
5     Feed ` st thy light 's flame with self-substan...
6                Making a famine where abundance lies ,
7      Thy self thy foe , to thy sweet self too cruel :
8       Thou that art now the world 's fresh ornament ,
9                 And only herald to the gaudy spring ,
10           Within thine own bud buriest thy content ,
11      And tender churl mak ` st waste in niggarding :
12           Pity the world , or else this glutton be ,
13    To eat the world 's due , by the grave and thee .
14          When forty winters shall besiege thy brow ,
15       And dig deep trenches in thy beauty 's field ,
16          Thy youth 's proud livery so gazed o

For thou art so possess 'd with murderous hate ,
That ` gainst thy self thou stick ` st not to conspire ,
Seeking that beauteous roof to ruinate
Which to repair should be thy chief desire .
O ! change thy thought , that I may change my mind :
Shall hate be fairer lodg 'd than gentle love ?
Be , as thy presence is , gracious and kind ,
Or to thyself at least kind-hearted prove :
Make thee another self for love of me ,
That beauty still may live in thine or thee .
As fast as thou shalt wane , so fast thou grow ` st ,
In one of thine , from that which thou departest ;
And that fresh blood which youngly thou bestow ` st ,
Thou mayst call thine when thou from youth convertest ,
Herein lives wisdom , beauty , and increase ;
Without this folly , age , and cold decay :
If all were minded so , the times should cease
And threescore year would make the world away .
Let those whom nature hath not made for store ,
Harsh , featureless , and rude , barrenly perish :
Look , whom she best endow 'd , sh

Is from the book of honour razed quite ,
And all the rest forgot for which he toil 'd :
Then happy I , that love and am belov 'd ,
Where I may not remove nor be remov 'd .
Lord of my love , to whom in vassalage
Thy merit hath my duty strongly knit ,
To thee I send this written embassage ,
To witness duty , not to show my wit :
Duty so great , which wit so poor as mine
May make seem bare , in wanting words to show it ,
But that I hope some good conceit of thine
In thy soul 's thought , all naked , will bestow it :
Till whatsoever star that guides my moving ,
Points on me graciously with fair aspect ,
And puts apparel on my tatter 'd loving ,
To show me worthy of thy sweet respect :
Then may I dare to boast how I do love thee ;
Till then , not show my head where thou mayst prove me .
Weary with toil , I haste me to my bed ,
The dear respose for limbs with travel tir 'd ;
But then begins a journey in my head
To work my mind , when body 's work 's expired :
For then my thoughts -- from far

These present-absent with swift motion slide .
For when these quicker elements are gone
In tender embassy of love to thee ,
My life , being made of four , with two alone
Sinks down to death , oppress 'd with melancholy ;
Until life 's composition be recur 'd
By those swift messengers return 'd from thee ,
Who even but now come back again , assur 'd ,
Of thy fair health , recounting it to me :
This told , I joy ; but then no longer glad ,
I send them back again , and straight grow sad .
Mine eye and heart are at a mortal war ,
How to divide the conquest of thy sight ;
Mine eye my heart thy picture 's sight would bar ,
My heart mine eye the freedom of that right .
My heart doth plead that thou in him dost lie , --
A closet never pierc 'd with crystal eyes --
But the defendant doth that plea deny ,
And says in him thy fair appearance lies .
To side this title is impannelled
A quest of thoughts , all tenants to the heart ;
And by their verdict is determined
The clear eye 's moiety , and th

The rich-proud cost of outworn buried age ;
When sometime lofty towers I see down-raz 'd ,
And brass eternal slave to mortal rage ;
When I have seen the hungry ocean gain
Advantage on the kingdom of the shore ,
And the firm soil win of the watery main ,
Increasing store with loss , and loss with store ;
When I have seen such interchange of state ,
Or state itself confounded , to decay ;
Ruin hath taught me thus to ruminate --
That Time will come and take my love away .
This thought is as a death which can not choose
But weep to have , that which it fears to lose .
Since brass , nor stone , nor earth , nor boundless sea ,
But sad mortality o'ersways their power ,
How with this rage shall beauty hold a plea ,
Whose action is no stronger than a flower ?
O ! how shall summer 's honey breath hold out ,
Against the wrackful siege of battering days ,
When rocks impregnable are not so stout ,
Nor gates of steel so strong but Time decays ?
O fearful meditation ! where , alack ,
Shall Time 's be

If thy sweet virtue answer not thy show !
They that have power to hurt , and will do none ,
That do not do the thing they most do show ,
Who , moving others , are themselves as stone ,
Unmoved , cold , and to temptation slow ;
They rightly do inherit heaven 's graces ,
And husband nature 's riches from expense ;
They are the lords and owners of their faces ,
Others , but stewards of their excellence .
The summer 's flower is to the summer sweet ,
Though to itself , it only live and die ,
But if that flower with base infection meet ,
The basest weed outbraves his dignity :
For sweetest things turn sourest by their deeds ;
Lilies that fester , smell far worse than weeds .
How sweet and lovely dost thou make the shame
Which , like a canker in the fragrant rose ,
Doth spot the beauty of thy budding name !
O ! in what sweets dost thou thy sins enclose .
That tongue that tells the story of thy days ,
Making lascivious comments on thy sport ,
Can not dispraise , but in a kind of praise ;
Nami

Or whether doth my mind , being crown 'd with you ,
Drink up the monarch 's plague , this flattery ?
Or whether shall I say , mine eye saith true ,
And that your love taught it this alchemy ,
To make of monsters and things indigest
Such cherubins as your sweet self resemble ,
Creating every bad a perfect best ,
As fast as objects to his beams assemble ?
O ! ` tis the first , ` tis flattery in my seeing ,
And my great mind most kingly drinks it up :
Mine eye well knows what with his gust is ` greeing ,
And to his palate doth prepare the cup :
If it be poison 'd , ` tis the lesser sin
That mine eye loves it and doth first begin .
Those lines that I before have writ do lie ,
Even those that said I could not love you dearer :
Yet then my judgment knew no reason why
My most full flame should afterwards burn clearer .
But reckoning Time , whose million 'd accidents
Creep in ` twixt vows , and change decrees of kings ,
Tan sacred beauty , blunt the sharp ` st intents ,
Divert strong minds to 

But my five wits nor my five senses can
Dissuade one foolish heart from serving thee ,
Who leaves unsway 'd the likeness of a man ,
Thy proud heart 's slave and vassal wretch to be :
Only my plague thus far I count my gain ,
That she that makes me sin awards me pain .
Love is my sin , and thy dear virtue hate ,
Hate of my sin , grounded on sinful loving :
O ! but with mine compare thou thine own state ,
And thou shalt find it merits not reproving ;
Or , if it do , not from those lips of thine ,
That have profan 'd their scarlet ornaments
And seal 'd false bonds of love as oft as mine ,
Robb 'd others ' beds ' revenues of their rents .
Be it lawful I love thee , as thou lov ` st those
Whom thine eyes woo as mine importune thee :
Root pity in thy heart , that , when it grows ,
Thy pity may deserve to pitied be .
If thou dost seek to have what thou dost hide ,
By self-example mayst thou be denied !
Lo , as a careful housewife runs to catch
One of her feather 'd creatures broke away ,
Sets

ModuleNotFoundError: No module named 'tensorflow'

In [1]:
import pandas as pd
import re
# Open a file: file
sent_table = pd.read_csv('data/corpus.txt', delimiter='|', usecols=['text'])
print(sent_table)


for i in sent_table.index.values:
    print(sent_table.iloc[i]['text']);
    #sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].replace('''O ''', '''Oh''')
    sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].replace('''`''', ''' \'''')
    sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].replace(''' ` ''', '''\'''')
    sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].replace(''' `''', '''\'''')
    sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].replace(''' \'''','''\'''')
    sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].replace('''\'d''','''ed''')
    #sent_table.iloc[i]['text'] = re.sub(r'''[^\w\d'\-\s]+''','',sent_table.iloc[i]['text']);
    sent_table.iloc[i]['text'] = sent_table.iloc[i]['text'].strip();
    
    
#%%
def uniq(input):
  output = []
  for x in input:
    if x not in output:
      output.append(x)
  return output

def recursive_len(item):
    if type(item) == list:
        return sum(recursive_len(subitem) for subitem in item)
    else:
        return 1



                                                   text
0           From fairest creatures we desire increase ,
1         That thereby beauty 's rose might never die ,
2             But as the riper should by time decease ,
3               His tender heir might bear his memory :
4      But thou , contracted to thine own bright eyes ,
5     Feed ` st thy light 's flame with self-substan...
6                Making a famine where abundance lies ,
7      Thy self thy foe , to thy sweet self too cruel :
8       Thou that art now the world 's fresh ornament ,
9                 And only herald to the gaudy spring ,
10           Within thine own bud buriest thy content ,
11      And tender churl mak ` st waste in niggarding :
12           Pity the world , or else this glutton be ,
13    To eat the world 's due , by the grave and thee .
14          When forty winters shall besiege thy brow ,
15       And dig deep trenches in thy beauty 's field ,
16          Thy youth 's proud livery so gazed o

When every private widow well may keep
By children 's eyes , her husband 's shape in mind :
Look ! what an unthrift in the world doth spend
Shifts but his place , for still the world enjoys it ;
But beauty 's waste hath in the world an end ,
And kept unused the user so destroys it .
No love toward others in that bosom sits
That on himself such murd ` rous shame commits .
For shame ! deny that thou bear ` st love to any ,
Who for thy self art so unprovident .
Grant , if thou wilt , thou art belov 'd of many ,
But that thou none lov ` st is most evident :
For thou art so possess 'd with murderous hate ,
That ` gainst thy self thou stick ` st not to conspire ,
Seeking that beauteous roof to ruinate
Which to repair should be thy chief desire .
O ! change thy thought , that I may change my mind :
Shall hate be fairer lodg 'd than gentle love ?
Be , as thy presence is , gracious and kind ,
Or to thyself at least kind-hearted prove :
Make thee another self for love of me ,
That beauty still m

For at a frown they in their glory die .
The painful warrior famoused for fight ,
After a thousand victories once foil 'd ,
Is from the book of honour razed quite ,
And all the rest forgot for which he toil 'd :
Then happy I , that love and am belov 'd ,
Where I may not remove nor be remov 'd .
Lord of my love , to whom in vassalage
Thy merit hath my duty strongly knit ,
To thee I send this written embassage ,
To witness duty , not to show my wit :
Duty so great , which wit so poor as mine
May make seem bare , in wanting words to show it ,
But that I hope some good conceit of thine
In thy soul 's thought , all naked , will bestow it :
Till whatsoever star that guides my moving ,
Points on me graciously with fair aspect ,
And puts apparel on my tatter 'd loving ,
To show me worthy of thy sweet respect :
Then may I dare to boast how I do love thee ;
Till then , not show my head where thou mayst prove me .
Weary with toil , I haste me to my bed ,
The dear respose for limbs with travel tir

All days are nights to see till I see thee ,
And nights bright days when dreams do show thee me .
If the dull substance of my flesh were thought ,
Injurious distance should not stop my way ;
For then despite of space I would be brought ,
From limits far remote , where thou dost stay .
No matter then although my foot did stand
Upon the farthest earth remov 'd from thee ;
For nimble thought can jump both sea and land ,
As soon as think the place where he would be .
But , ah ! thought kills me that I am not thought ,
To leap large lengths of miles when thou art gone ,
But that so much of earth and water wrought ,
I must attend time 's leisure with my moan ;
Receiving nought by elements so slow
But heavy tears , badges of either 's woe .
The other two , slight air , and purging fire
Are both with thee , wherever I abide ;
The first my thought , the other my desire ,
These present-absent with swift motion slide .
For when these quicker elements are gone
In tender embassy of love to thee ,
M

So true a fool is love , that in your will ,
Though you do anything , he thinks no ill .
That god forbid , that made me first your slave ,
I should in thought control your times of pleasure ,
Or at your hand the account of hours to crave ,
Being your vassal , bound to stay your leisure !
O ! let me suffer , being at your beck ,
The imprison 'd absence of your liberty ;
And patience , tame to sufferance , bide each check ,
Without accusing you of injury .
Be where you list , your charter is so strong
That you yourself may privilage your time
To what you will ; to you it doth belong
Yourself to pardon of self-doing crime .
I am to wait , though waiting so be hell ,
Not blame your pleasure be it ill or well .
If there be nothing new , but that which is
Hath been before , how are our brains beguil 'd ,
Which labouring for invention bear amiss
The second burthen of a former child !
O ! that record could with a backward look ,
Even of five hundred courses of the sun ,
Show me your image in s

Thus do I pine and surfeit day by day ,
Or gluttoning on all , or all away .
Why is my verse so barren of new pride ,
So far from variation or quick change ?
Why with the time do I not glance aside
To new-found methods , and to compounds strange ?
Why write I still all one , ever the same ,
And keep invention in a noted weed ,
That every word doth almost tell my name ,
Showing their birth , and where they did proceed ?
O ! know sweet love I always write of you ,
And you and love are still my argument ;
So all my best is dressing old words new ,
Spending again what is already spent :
For as the sun is daily new and old ,
So is my love still telling what is told .
Thy glass will show thee how thy beauties wear ,
Thy dial how thy precious minutes waste ;
These vacant leaves thy mind 's imprint will bear ,
And of this book , this learning mayst thou taste .
The wrinkles which thy glass will truly show
Of mouthed graves will give thee memory ;
Thou by thy dial 's shady stealth mayst know
Ti

And every humour hath his adjunct pleasure ,
Wherein it finds a joy above the rest :
But these particulars are not my measure ,
All these I better in one general best .
Thy love is better than high birth to me ,
Richer than wealth , prouder than garments ' costs ,
Of more delight than hawks and horses be ;
And having thee , of all men 's pride I boast :
Wretched in this alone , that thou mayst take
All this away , and me most wretchcd make .
But do thy worst to steal thyself away ,
For term of life thou art assured mine ;
And life no longer than thy love will stay ,
For it depends upon that love of thine .
Then need I not to fear the worst of wrongs ,
When in the least of them my life hath end .
I see a better state to me belongs
Than that which on thy humour doth depend :
Thou canst not vex me with inconstant mind ,
Since that my life on thy revolt doth lie .
O ! what a happy title do I find ,
Happy to have thy love , happy to die !
But what 's so blessed-fair that fears no blot ?
Tho

Finding the first conceit of love there bred ,
Where time and outward form would show it dead .
O ! never say that I was false of heart ,
Though absence seem 'd my flame to qualify ,
As easy might I from my self depart
As from my soul which in thy breast doth lie :
That is my home of love : if I have rang 'd ,
Like him that travels , I return again ;
Just to the time , not with the time exchang 'd ,
So that myself bring water for my stain .
Never believe though in my nature reign 'd ,
All frailties that besiege all kinds of blood ,
That it could so preposterously be stain 'd ,
To leave for nothing all thy sum of good ;
For nothing this wide universe I call ,
Save thou , my rose , in it thou art my all .
Alas ! ` tis true , I have gone here and there ,
And made my self a motley to the view ,
Gor 'd mine own thoughts , sold cheap what is most dear ,
Made old offences of affections new ;
Most true it is , that I have look 'd on truth
Askance and strangely ; but , by all above ,
These blen

Enjoy 'd no sooner but despised straight ;
Past reason hunted ; and no sooner had ,
Past reason hated , as a swallow 'd bait ,
On purpose laid to make the taker mad :
Mad in pursuit and in possession so ;
Had , having , and in quest , to have extreme ;
A bliss in proof , -- and prov 'd , a very woe ;
Before , a joy propos 'd ; behind a dream .
All this the world well knows ; yet none knows well
To shun the heaven that leads men to this hell .
My mistress ' eyes are nothing like the sun ;
Coral is far more red , than her lips red :
If snow be white , why then her breasts are dun ;
If hairs be wires , black wires grow on her head .
I have seen roses damask 'd , red and white ,
But no such roses see I in her cheeks ;
And in some perfumes is there more delight
Than in the breath that from my mistress reeks .
I love to hear her speak , yet well I know
That music hath a far more pleasing sound :
I grant I never saw a goddess go , --
My mistress , when she walks , treads on the ground :
And y

When I against myself with thee partake ?
Do I not think on thee , when I forgot
Am of my self , all tyrant , for thy sake ?
Who hateth thee that I do call my friend ,
On whom frown ` st thou that I do fawn upon ,
Nay , if thou lour ` st on me , do I not spend
Revenge upon myself with present moan ?
What merit do I in my self respect ,
That is so proud thy service to despise ,
When all my best doth worship thy defect ,
Commanded by the motion of thine eyes ?
But , love , hate on , for now I know thy mind ;
Those that can see thou lov ` st , and I am blind .
O ! from what power hast thou this powerful might ,
With insufficiency my heart to sway ?
To make me give the lie to my true sight ,
And swear that brightness doth not grace the day ?
Whence hast thou this becoming of things ill ,
That in the very refuse of thy deeds
There is such strength and warrantise of skill ,
That , in my mind , thy worst all best exceeds ?
Who taught thee how to make me love thee more ,
The more I hear and se

In [68]:
import nltk
import numpy as np
import time
from datetime import datetime
from enum import Enum 
import json
import pprint
num_figs=10
max_fig_length=18
base_path = 'data/rhet/repitition/'
base_path_output = 'data/rhet/output/'
corpus = 'sonnets'

fig_names = ["Anadiplosis", "Anaphora", "Antimetabole", "Conduplicatio", "Epanalepsis", "Epistrophe", "Epizeuxis", "Ploce", "Polysyndeton", "Symploce"]

class RHETORICAL_FIGURES(Enum):
    ANADIPLOSIS = 0
    ANAPHORA = 1
    ANTIMETABOLE = 2
    CONDUPLICATIO = 3
    EPANALEPSIS = 4
    EPISTROPHE = 5
    EPIZEUXIS = 6
    PLOCE = 7
    POLYSYNDETON = 8
    SYMPLOCE = 9
    
#FIGURE_DESCRIPTION[RHETORICAL_FIGURES['ANADIPLOSIS'].name].value
class FIGURE_DESCRIPTION(Enum):
    ANADIPLOSIS = 'Repetition of the ending word or phrase from the previous clause at the beginning of the next.'
    ANAPHORA = 'Repetition of a word or phrase at the beginning of successive phrases or clauses.'
    ANTIMETABOLE = 'Repetition of words in reverse grammatical order.'
    CONDUPLICATIO = 'The repetition of a word or phrase.'
    EPANALEPSIS = 'Repetition at the end of a clause of the word or phrase that began it.'
    EPISTROPHE = 'Repetition of the same word or phrase at the end of successive clauses.'
    EPIZEUXIS = 'Repetition of a word or phrase with no others between.'
    PLOCE = 'The repetition of word in a short span of text for rhetorical emphasis.'
    POLYSYNDETON = '"Excessive" repetition of conjunctions between clauses.'
    SYMPLOCE = 'Repetition of a word or phrase at the beginning, and of another at the end, of successive clauses.'

class FigureInfo:

    def __init__(self, unique_id, orig_text, pos_template, repitition_template):
        self.unique_id = unique_id
        self.orig_text = orig_text
        self.pos_template = pos_template
        self.repitition_template = repitition_template
    
    def set_props(self, num_lines,num_tokens,num_rep_groups,num_tot_reps,fig_type):
        self.num_lines = num_lines
        self.num_tokens = num_tokens
        self.num_rep_groups = num_rep_groups
        self.num_tot_reps = num_tot_reps
        self.fig_type = fig_type
    
    def set_orig_rep_words(self, orig_rep_words):
        self.orig_rep_words = orig_rep_words
    
    def get_orig_rep_words(self):
        return self.orig_rep_words
    
    def get_unique_id(self):
        return self.unique_id
    
    def get_orig_text(self):
        return self.orig_text
    
    def get_pos_template(self):
        return self.pos_template
    
    def get_repitition_template(self):
        return self.repitition_template
    
    def get_num_lines(self):
        return self.num_lines
    
    def get_num_tokens(self):
        return self.num_tokens
    
    def get_num_rep_groups(self):
        return self.num_rep_groups
    
    def get_num_tot_reps(self):
        return self.num_tot_reps
    
    def get_fig_type(self):
        return self.fig_type
    
    def get_fig_desc(self):
        return FIGURE_DESCRIPTION[self.get_fig_type()].value
    
    def to_string(self):
        pp = pprint.PrettyPrinter(indent=4, width=160)
        return pprint.pformat(self.__dict__) + "\n"
    
    def print(self):
        pp = pprint.PrettyPrinter(indent=4, width=160)
        pp.pprint(self.__dict__)
        print("\n")

    
    
    
fig_descs = \
    [
        "Repetition of the ending word or phrase from the previous clause at the beginning of the next.",
        "Repetition of a word or phrase at the beginning of successive phrases or clauses.",
        "Repetition of words in reverse grammatical order.",
        "The repetition of a word or phrase.",
        "Repetition at the end of a clause of the word or phrase that began it.",
        "Repetition of the same word or phrase at the end of successive clauses.",
        "Repetition of a word or phrase with no others between.",
        "The repetition of word in a short span of text for rhetorical emphasis.",
        "\"Excessive\" repetition of conjunctions between clauses.",
        "Repetition of a word or phrase at the beginning, and of another at the end, of successive clauses."
    ]

rep_figs = ["sonnet_anadiplosis.csv", "sonnet_anaphora.csv", "sonnet_antimetabole.csv",
"sonnet_conduplicatio.csv", 
            "sonnet_epanalepsis.csv", "sonnet_epistrophe.csv", "sonnet_epizeuxis.csv",
"sonnet_ploce.csv", 
            "sonnet_polysyndeton.csv", "sonnet_symploce.csv"]
unique_id = 0
fig_dict = dict()
for nfig in range(0,num_figs):
    cur_fig_name = fig_names[nfig]
    cur_fig_desc = fig_descs[nfig]
    cur_fig_rhet_file = rep_figs[nfig]

    rhet_table = pd.read_csv(base_path+cur_fig_rhet_file)
    #print(rhet_table)
    fig_ids = rhet_table.figure_id.unique()
    #print(fig_ids)
    for id in fig_ids:
       figure = rhet_table.loc[rhet_table['figure_id'] == id]
       rep_word = []
       sentence_ids = figure['sentence_id'].values
       sentence_ids = uniq(sentence_ids)
       if(not sorted(sentence_ids) == list(range(min(sentence_ids), max(sentence_ids)+1))): continue 

       fig_words = uniq([x.lower().replace('''\'d''', '''ed''') for x in list(figure['word'].values)])
       fig_words_dict = dict([[fig_words[y-1], y] for y in range(1, len(fig_words)+1)])
       #print(fig_words_dict)
        
       orig_text = [ nltk.word_tokenize(sent_table.iloc[i]['text']) for i in sentence_ids]
       
       #for l in range(0,len(roman)):
           #del orig_text[roman[l][0]][roman[l][1]]
    
       orig_text = [[orig_text[i][j].lower() for j in range(0,len(orig_text[i]))] for i in range(0,len(orig_text))]
       if(recursive_len(orig_text) > 1):
           tagged = [nltk.pos_tag(orig_text[i]) for i in  range(0,len(orig_text))]
           #print(orig_text)
           pos_templates = [[tagged[i][j][1] for j in range(0,len(tagged[i]))] for i in range(0,len(tagged))]
           #print(pos_templates)
           
           assert(np.shape(orig_text) == np.shape(pos_templates))
           fig_words_count_dict = dict([[fig_words[y], 0] for y in range(0, len(fig_words))])
        
           rep_templates = [['0' for j in range(0,len(orig_text[i]))] for i in range(0,len(orig_text))]
           for j in range(0,len(orig_text)):
               for k in range(0,len(orig_text[j])):
                   wordc = orig_text[j][k]
                   for word_rep in list(fig_words_dict.keys()):
                       if(word_rep in wordc):
                           fig_words_count_dict[word_rep] = fig_words_count_dict[word_rep]+1
                           rep_templates[j][k] = str(fig_words_dict[word_rep]) + str(fig_words_count_dict[word_rep])
           
           unique_id = unique_id+1
           #print("unique_id: " + str(unique_id))
           #print("num_lines: " + str(len(sentence_ids)))
           #print("num_tokens: " + str(recursive_len(orig_text)))
           #print("num_rep_groups: " + str(len(fig_words_dict.keys())))
           #print("num_tot_reps: " + str(sum(fig_words_count_dict.values())))
           #print("fig_type: " + str(RHETORICAL_FIGURES[cur_fig_name.upper()].name)+ "\n")
           #print("Orig. Text:\n")
           orig_line = ", ".join([' '.join(orig_text[i]) for i in range(0,len(orig_text))])
           fig_info = FigureInfo(unique_id, [[sent_table.iloc[i]['text']] for i in sentence_ids], pos_templates, rep_templates)
           fig_info.set_props(len(sentence_ids), recursive_len(orig_text), len(fig_words_dict.keys()), sum(fig_words_count_dict.values()), RHETORICAL_FIGURES[cur_fig_name.upper()].name)
           fig_info.set_orig_rep_words([k  for  k in fig_words_dict.keys()])
           fig_dict[unique_id] = fig_info
           #print(orig_text)
           #print(rep_templates)
           #print(pos_templates)
           #print("Rep. Words:\n")
           #print(fig_words_dict)
           #print("Rep. Words Count:\n")
           #print(fig_words_count_dict)
           #print("POS Template:\n")
           #print(pos_templates)
           #print("Rep Template:\n")
           #print(rep_templates)
           
  
   #for index, row in figure.iterrows():
    
print(fig_dict.values)

<built-in method values of dict object at 0x113914d38>


In [63]:
for i in range(0,len(fig_names)):
    print(fig_names[i].upper() + " = " + "\'%s\'" % fig_descs[i])

ANADIPLOSIS = 'Repetition of the ending word or phrase from the previous clause at the beginning of the next.'
ANAPHORA = 'Repetition of a word or phrase at the beginning of successive phrases or clauses.'
ANTIMETABOLE = 'Repetition of words in reverse grammatical order.'
CONDUPLICATIO = 'The repetition of a word or phrase.'
EPANALEPSIS = 'Repetition at the end of a clause of the word or phrase that began it.'
EPISTROPHE = 'Repetition of the same word or phrase at the end of successive clauses.'
EPIZEUXIS = 'Repetition of a word or phrase with no others between.'
PLOCE = 'The repetition of word in a short span of text for rhetorical emphasis.'
POLYSYNDETON = '"Excessive" repetition of conjunctions between clauses.'
SYMPLOCE = 'Repetition of a word or phrase at the beginning, and of another at the end, of successive clauses.'


In [73]:
#from py_files.Limericks import Limerick_Generate
#lg = Limerick_Generate(model_dir='/gpt2/models/345M',model_name='345M')
#n=200
    
single_line = {k: v for k, v in fig_dict.items() if v.get_num_lines() == 1 }
print(len(single_line))
n=50

for k, v in single_line.items():
    string = v.get_pos_template
    #prompt = lg.enc.encode(v.get_orig_text()[0])
    rep_template = v.get_repitition_template()[0]
    pos_templates = v.get_pos_template()[0]
    banned_set = v.get_orig_rep_words()
    
    try:
        print(string)
        print(banned_set)
        #new_sentences = lg.gen_line_gpt_cc(cctemplate=rep_template, w=None, encodes=prompt, default_template=pos_template, banned_set=banned_set,search_space=n)
        #sents = [new_sentences[j][0] for j in range(len(new_sentences))]
        #sent_done = [' '.join(sents[j]) for j in range(len(new_sentences))]
        #print(" \n".join(sent_done))

    except Exception:
        pass
    #f.write("\n-----------------------------------------------------------------------------------------------------------------------------------------------------\n")     

    v.print()

240
{   'fig_type': 'ANADIPLOSIS',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 12,
    'num_tot_reps': 2,
    'orig_rep_words': ['one'],
    'orig_text': [['Who , all in one , one pleasing note do sing :']],
    'pos_template': [['WP', ',', 'DT', 'IN', 'CD', ',', 'CD', 'NN', 'NN', 'VBP', 'NN', ':']],
    'repitition_template': [['0', '0', '0', '0', '11', '0', '12', '0', '0', '0', '0', '0']],
    'unique_id': 1}


None
['one']
{   'fig_type': 'ANADIPLOSIS',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 12,
    'num_tot_reps': 2,
    'orig_rep_words': ['one'],
    'orig_text': [['Who , all in one , one pleasing note do sing :']],
    'pos_template': [['WP', ',', 'DT', 'IN', 'CD', ',', 'CD', 'NN', 'NN', 'VBP', 'NN', ':']],
    'repitition_template': [['0', '0', '0', '0', '11', '0', '12', '0', '0', '0', '0', '0']],
    'unique_id': 1}


{   'fig_type': 'ANADIPLOSIS',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 10,
    'num_tot_reps': 2

    'num_rep_groups': 1,
    'num_tokens': 11,
    'num_tot_reps': 2,
    'orig_rep_words': ['thy'],
    'orig_text': [['Thy looks with me , thy heart in other place :']],
    'pos_template': [['JJ', 'VBZ', 'IN', 'PRP', ',', 'JJ', 'NN', 'IN', 'JJ', 'NN', ':']],
    'repitition_template': [['11', '0', '0', '0', '0', '12', '0', '0', '0', '0', '0']],
    'unique_id': 87}


{   'fig_type': 'ANAPHORA',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 11,
    'num_tot_reps': 2,
    'orig_rep_words': ['what'],
    'orig_text': [['What freezings have I felt , what dark days seen !']],
    'pos_template': [['WP', 'NNS', 'VBP', 'VBN', 'NNS', ',', 'WP', 'JJ', 'NNS', 'VBN', '.']],
    'repitition_template': [['11', '0', '0', '0', '0', '0', '12', '0', '0', '0', '0']],
    'unique_id': 91}


None
['what']
{   'fig_type': 'ANAPHORA',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 11,
    'num_tot_reps': 2,
    'orig_rep_words': ['what'],
    'orig_text': [['What freezing

    'unique_id': 211}


None
['to', 'time', "'s"]
{   'fig_type': 'CONDUPLICATIO',
    'num_lines': 1,
    'num_rep_groups': 3,
    'num_tokens': 12,
    'num_tot_reps': 6,
    'orig_rep_words': ['to', 'time', "'s"],
    'orig_text': [["As subject to Time's love or to Time's hate ,"]],
    'pos_template': [['IN', 'JJ', 'TO', 'NN', 'POS', 'NN', 'CC', 'TO', 'NN', 'POS', 'NN', ',']],
    'repitition_template': [['0', '0', '11', '21', '31', '0', '0', '12', '22', '32', '0', '0']],
    'unique_id': 211}


{   'fig_type': 'CONDUPLICATIO',
    'num_lines': 1,
    'num_rep_groups': 2,
    'num_tokens': 10,
    'num_tot_reps': 4,
    'orig_rep_words': ['thou', 'art'],
    'orig_text': [['Thou art as tyrannous , so as thou art ,']],
    'pos_template': [['JJ', 'NN', 'IN', 'JJ', ',', 'RB', 'IN', 'JJ', 'NN', ',']],
    'repitition_template': [['11', '21', '0', '0', '0', '0', '0', '12', '22', '0']],
    'unique_id': 216}


None
['thou', 'art']
{   'fig_type': 'CONDUPLICATIO',
    'num_lines': 1,
   

    'num_rep_groups': 1,
    'num_tokens': 10,
    'num_tot_reps': 2,
    'orig_rep_words': ['action'],
    'orig_text': [['Is lust in action : and till action , lust']],
    'pos_template': [['VBZ', 'JJ', 'IN', 'NN', ':', 'CC', 'VB', 'NN', ',', 'NN']],
    'repitition_template': [['0', '0', '0', '11', '0', '0', '0', '12', '0', '0']],
    'unique_id': 281}


None
['action']
{   'fig_type': 'EPISTROPHE',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 10,
    'num_tot_reps': 2,
    'orig_rep_words': ['action'],
    'orig_text': [['Is lust in action : and till action , lust']],
    'pos_template': [['VBZ', 'JJ', 'IN', 'NN', ':', 'CC', 'VB', 'NN', ',', 'NN']],
    'repitition_template': [['0', '0', '0', '11', '0', '0', '0', '12', '0', '0']],
    'unique_id': 281}


{   'fig_type': 'EPISTROPHE',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 11,
    'num_tot_reps': 2,
    'orig_rep_words': ['red'],
    'orig_text': [['Coral is far more red , than her lips red

    'orig_text': [['Excusing thy sins more than thy sins are ;']],
    'pos_template': [['VBG', 'JJ', 'NNS', 'RBR', 'IN', 'JJ', 'NNS', 'VBP', ':']],
    'repitition_template': [['0', '0', '11', '0', '0', '0', '12', '0', '0']],
    'unique_id': 376}


None
['sins']
{   'fig_type': 'PLOCE',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 9,
    'num_tot_reps': 2,
    'orig_rep_words': ['sins'],
    'orig_text': [['Excusing thy sins more than thy sins are ;']],
    'pos_template': [['VBG', 'JJ', 'NNS', 'RBR', 'IN', 'JJ', 'NNS', 'VBP', ':']],
    'repitition_template': [['0', '0', '11', '0', '0', '0', '12', '0', '0']],
    'unique_id': 376}


{   'fig_type': 'PLOCE',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 11,
    'num_tot_reps': 2,
    'orig_rep_words': ['mine'],
    'orig_text': [['As thou being mine , mine is thy good report .']],
    'pos_template': [['IN', 'NN', 'VBG', 'NN', ',', 'NN', 'VBZ', 'JJ', 'JJ', 'NN', '.']],
    'repitition_template': [['

{   'fig_type': 'PLOCE',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 11,
    'num_tot_reps': 2,
    'orig_rep_words': ['colour'],
    'orig_text': [["' Truth needs no colour , with his colour fixed ;"]],
    'pos_template': [['POS', 'NN', 'VBZ', 'DT', 'NN', ',', 'IN', 'PRP$', 'NN', 'VBD', ':']],
    'repitition_template': [['0', '0', '0', '0', '11', '0', '0', '0', '12', '0', '0']],
    'unique_id': 518}


{   'fig_type': 'PLOCE',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 10,
    'num_tot_reps': 2,
    'orig_rep_words': ['beauty'],
    'orig_text': [["Beauty no pencil , beauty's truth to lay ;"]],
    'pos_template': [['NN', 'DT', 'NN', ',', 'NN', 'POS', 'NN', 'TO', 'VB', ':']],
    'repitition_template': [['11', '0', '0', '0', '12', '0', '0', '0', '0', '0']],
    'unique_id': 519}


None
['beauty']
{   'fig_type': 'PLOCE',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 10,
    'num_tot_reps': 2,
    'orig_rep_words': ['beauty'],
 



None
['red']
{   'fig_type': 'PLOCE',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 11,
    'num_tot_reps': 2,
    'orig_rep_words': ['red'],
    'orig_text': [['Coral is far more red , than her lips red :']],
    'pos_template': [['NN', 'VBZ', 'RB', 'RBR', 'JJ', ',', 'IN', 'PRP$', 'NNS', 'JJ', ':']],
    'repitition_template': [['0', '0', '0', '0', '11', '0', '0', '0', '0', '12', '0']],
    'unique_id': 571}


{   'fig_type': 'PLOCE',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 12,
    'num_tot_reps': 2,
    'orig_rep_words': ['wires'],
    'orig_text': [['If hairs be wires , black wires grow on her head .']],
    'pos_template': [['IN', 'NNS', 'VB', 'NNS', ',', 'JJ', 'NNS', 'VBP', 'IN', 'PRP$', 'NN', '.']],
    'repitition_template': [['0', '0', '0', '11', '0', '0', '12', '0', '0', '0', '0', '0']],
    'unique_id': 572}


None
['wires']
{   'fig_type': 'PLOCE',
    'num_lines': 1,
    'num_rep_groups': 1,
    'num_tokens': 12,
    'num_tot_reps':

In [71]:

fig_and_desc = [e.name + ": " + d.value for e,d in zip(RHETORICAL_FIGURES,FIGURE_DESCRIPTION)]
fig_and_desc.insert(0, "FIGURE: Description\n")
for fd in fig_and_desc:
    print(fd)

FIGURE: Description

ANADIPLOSIS: Repetition of the ending word or phrase from the previous clause at the beginning of the next.
ANAPHORA: Repetition of a word or phrase at the beginning of successive phrases or clauses.
ANTIMETABOLE: Repetition of words in reverse grammatical order.
CONDUPLICATIO: The repetition of a word or phrase.
EPANALEPSIS: Repetition at the end of a clause of the word or phrase that began it.
EPISTROPHE: Repetition of the same word or phrase at the end of successive clauses.
EPIZEUXIS: Repetition of a word or phrase with no others between.
PLOCE: The repetition of word in a short span of text for rhetorical emphasis.
POLYSYNDETON: "Excessive" repetition of conjunctions between clauses.
SYMPLOCE: Repetition of a word or phrase at the beginning, and of another at the end, of successive clauses.
